# Spider the internet to find Deep Learning Blogs
Indebted to https://www.py4e.com/lessons

In [1]:
import sqlite3
import urllib.error
import ssl
from urllib.parse import urljoin
from urllib.parse import urlparse
from urllib.request import urlopen
from urllib.request import Request
from bs4 import BeautifulSoup

In [2]:
# Ignore SSL certificate errors
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

In [3]:
import os
filenames = ['spider.sqlite', 'spider.sqlite-journal']
for filename in filenames:
    if os.path.exists(filename):
        os.remove(filename)

In [4]:
def is_dl_content(paragraphs):
    DL_buzzwords = [' cnn ',' deep rl ',' gru ',' neural machine translation ',' deep learning ', ' rnn ', ' neural networks ',' lstm ', ' computer vision ', ' nlp ', ' deep reinforcement learning ', ' image classification ', ' imagenet ', ' resnet ', ' mobilenet ','memory networks', 'adversarial networks']
    for paragraph in paragraphs:
        if any( k in paragraph.text.lower() for k in DL_buzzwords):
            return True 
    return False

In [5]:
very_first = "https://blog.floydhub.com/spinning-up-with-deep-reinforcement-learning/" #random blog to get the crawling started

In [6]:
conn = sqlite3.connect('spider.sqlite')
cur = conn.cursor()

In [7]:
cur.execute('''CREATE TABLE IF NOT EXISTS Pages
    (id INTEGER PRIMARY KEY, url TEXT UNIQUE, html TEXT,author TEXT,title TEXT,
     error INTEGER, old_rank REAL, new_rank REAL)''')

cur.execute('''CREATE TABLE IF NOT EXISTS Links
    (from_id INTEGER, to_id INTEGER)''')

cur.execute('''CREATE TABLE IF NOT EXISTS Webs (url TEXT UNIQUE)''')

In [8]:
# Check to see if we are already in progress...
cur.execute('SELECT id,url FROM Pages WHERE html is NULL and error is NULL ORDER BY RANDOM() LIMIT 1')
row = cur.fetchone()
if row is not None:
    print("Restarting existing crawl.  Remove spider.sqlite to start a fresh crawl.")
else :
    starturl = input('Enter web url or enter: ')
    if ( len(starturl) < 1 ) : starturl = very_first
    if ( starturl.endswith('/') ) : starturl = starturl[:-1]
    web = starturl
    if ( starturl.endswith('.htm') or starturl.endswith('.html') ) :
        pos = starturl.rfind('/')
        web = starturl[:pos]
    print("web :",web)
    if ( len(web) > 1 ) :
        cur.execute('INSERT OR IGNORE INTO Webs (url) VALUES ( ? )', ( web, ) )
        cur.execute('INSERT OR IGNORE INTO Pages (url, html, new_rank) VALUES ( ?, NULL, 1.0 )', ( starturl, ) )
        conn.commit()
        ##test##
        #cur1 = conn.cursor()
        #cur1.execute('SELECT * FROM Pages')
        #print(cur1.fetchone())
# Get the current webs
cur.execute('''SELECT url FROM Webs''')
webs = list()
for row in cur:
    #print("row:",row)
    webs.append(str(row[0]))

print(webs)

Enter web url or enter: 
web : https://blog.floydhub.com/spinning-up-with-deep-reinforcement-learning
['https://blog.floydhub.com/spinning-up-with-deep-reinforcement-learning']


In [9]:
many = 0
while True:
    if ( many < 1 ) :
        sval = input('How many pages:')
        if ( len(sval) < 1 ) : break
        many = int(sval)
    many = many - 1

    cur.execute('SELECT id,url FROM Pages WHERE html is NULL and error is NULL ORDER BY RANDOM() LIMIT 1')
    #print("pass")
    try:
        row = cur.fetchone()
        # print row
        fromid = row[0]
        url = row[1]
    except Exception as e:
        print('No unretrieved HTML pages found')
        print(e)
        many = 0
        break

    print(fromid, url, end=' ')

    # If we are retrieving this page, there should be no links from it
    cur.execute('DELETE from Links WHERE from_id=?', (fromid, ) )
    try:
        #document = urlopen(url, context=ctx)
        req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
        document = urlopen(req)
        html = document.read()
        if document.getcode() != 200 :
            print("Error on page: ",document.getcode())
            cur.execute('UPDATE Pages SET error=? WHERE url=?', (document.getcode(), url) )

        if 'text/html' != document.info().get_content_type() :
            print("Ignore non text/html page")
            cur.execute('DELETE FROM Pages WHERE url=?', ( url, ) )
            cur.execute('UPDATE Pages SET error=0 WHERE url=?', (url, ) )
            conn.commit()
            continue

        #print('('+str(len(html))+')', end=' ')

        soup = BeautifulSoup(html, "html.parser")
        #print("soup",soup('p'))
        #print("dl content :",is_dl_content(soup('p')))
        if not is_dl_content(soup('p')):
            print("Not Dl content",url)
            cur.execute('UPDATE Pages SET error=-2 WHERE url=?', (url, ) )
            conn.commit()
            continue
            
    except KeyboardInterrupt:
        print('')
        print('Program interrupted by user...')
        break
    except Exception as e:
        print("Unable to retrieve or parse page")
        print(e)
        cur.execute('UPDATE Pages SET error=-1 WHERE url=?', (url, ) )
        conn.commit()
        continue

    cur.execute('INSERT OR IGNORE INTO Pages (url, html, new_rank) VALUES ( ?, NULL, 1.0 )', ( url, ) )
    cur.execute('UPDATE Pages SET html=? WHERE url=?', (memoryview(html), url ) )
    conn.commit()

    # Retrieve all of the anchor tags
    tags = soup('a')
    #print("tags: ",tags)
    count = 0
    for tag in tags:
        href = tag.get('href', None)
        if ( href is None ) : continue
        # Resolve relative references like href="/contact"
        up = urlparse(href)
        #print("up.scheme : ",up.scheme)
        if ( len(up.scheme) < 1 ) :
            #print("up :",up)
            href = urljoin(url, href)
            #print("after join :",href)
        ipos = href.find('#')
        if ( ipos > 1 ) : href = href[:ipos]
         
        #if (len(up.path) < 1 ) : 
            #print("landing page",url)
            #continue
            
        if (href.startswith('https://arxiv.org') 
        or href.startswith('http://arxiv.org')  
        or href.startswith('https://github.com')
        or href.startswith('http://github.com')
        or href.startswith('https://twitter.com')
        or href.startswith('http://www.twitter.com')
        or href.startswith('https://www.twitter.com')
        or href.startswith('http://twitter.com')
        or href.startswith('https://www.facebook.com')
        or href.startswith('http://www.facebook.com')
        or href.startswith('http://www.nature.com')
        or href.startswith('javascript')
        or href.startswith('https://gist.github.com')
        or href.startswith('http://mts')
        or ('docs.google.com' in href)
        or ('?' in href)
        or ('@' in href)
        or ('.edu' in href)
        or ('wikipedia.org' in href)):
        #or ('keras.io' in href)):
            continue
        if ('.org' in href) and ('tutorial' not in href):
            continue
        if ( href.endswith('.png') or href.endswith('.jpg') or href.endswith('.gif') or href.endswith('.pdf') or href.endswith('.xml')) : continue
        if ( href.endswith('/') ) : href = href[:-1]
        #print("href final:",href)
        if ( len(href) < 1 ) : continue

		# Check if the URL is in any of the webs
        found = False
        for web in webs:
            if ( href.startswith(web) ) :
                #print("web inside :",web)
                #print("startswith :",href.startswith(web))
                found = True
                break
        
        title = href.split('/')[-1]
        #if not found : continue
       
        cur.execute('INSERT OR IGNORE INTO Pages (url, title, html, new_rank) VALUES ( ?, ?, NULL, 1.0 )', ( href,title, ) )
        count = count + 1
        conn.commit()

        cur.execute('SELECT id FROM Pages WHERE url=? LIMIT 1', ( href, ))
        #print("href :",href)
        try:
            row = cur.fetchone()
            toid = row[0]
        except Exception as e:
            print('Could not retrieve id')
            print(e)
            continue
        # print fromid, toid
        cur.execute('INSERT OR IGNORE INTO Links (from_id, to_id) VALUES ( ?, ? )', ( fromid, toid ) )


    #print(count)

cur.close()

How many pages:10000
1 https://blog.floydhub.com/spinning-up-with-deep-reinforcement-learning 8 https://spinningup.openai.com/en/latest/spinningup/spinningup.html 13 https://www.floydhub.com/projects/create Not Dl content https://www.floydhub.com/projects/create
41 https://spinningup.openai.com/en/latest/algorithms/ppo.html Not Dl content https://spinningup.openai.com/en/latest/algorithms/ppo.html
37 https://spinningup.openai.com/en/latest/spinningup/exercises.html 17 https://medium.com/mlreview/speeding-up-dqn-on-pytorch-solving-pong-in-30-minutes-81a1bd2dff55 11 https://lilianweng.github.io/lil-log/2018/04/08/policy-gradient-algorithms.html 90 http://kvfrans.com/a-intuitive-explanation-of-natural-gradient-descent 82 https://lilianweng.github.io/lil-log/tags.html Not Dl content https://lilianweng.github.io/lil-log/tags.html
87 https://danieltakeshi.github.io/2017/03/28/going-deeper-into-reinforcement-learning-fundamentals-of-policy-gradients Not Dl content https://danieltakeshi.github

750 https://courses.nvidia.com/courses/course-v1:DLI+A-AC-01+V1/about Not Dl content https://courses.nvidia.com/courses/course-v1:DLI+A-AC-01+V1/about
909 https://www.boozallen.com/e/employees/employee-portal.html Not Dl content https://www.boozallen.com/e/employees/employee-portal.html
881 https://www.boozallen.com/about/diversity-and-inclusion.html Not Dl content https://www.boozallen.com/about/diversity-and-inclusion.html
742 https://courses.nvidia.com/courses/course-v1:DLI+L-FX-04+V1/about Not Dl content https://courses.nvidia.com/courses/course-v1:DLI+L-FX-04+V1/about
408 http://www.nvidia.com/object/drive-automotive-technology.html 263 https://wach.com/news/auto-matters/uber-approved-to-resume-autonomous-car-tests-in-pittsburgh Not Dl content https://wach.com/news/auto-matters/uber-approved-to-resume-autonomous-car-tests-in-pittsburgh
575 https://blogs.nvidia.com/blog/category/pro-graphics Not Dl content https://blogs.nvidia.com/blog/category/pro-graphics
122 http://kvfrans.com/p

1592 https://diuf.unifr.ch/main/hisdoc/taxonomy/term/2 Unable to retrieve or parse page
HTTP Error 403: Forbidden
134 http://kvfrans.com/colorizing-the-draw-model Not Dl content http://kvfrans.com/colorizing-the-draw-model
465 https://clarifai.com/models/face-embedding-image-recognition-model-d02b4508df58432fbb84e800597b8959 Not Dl content https://clarifai.com/models/face-embedding-image-recognition-model-d02b4508df58432fbb84e800597b8959
940 https://openzeka.com/iletisim Not Dl content https://openzeka.com/iletisim
882 https://www.boozallen.com/about/diversity-and-inclusion/veterans-and-military-families.html Not Dl content https://www.boozallen.com/about/diversity-and-inclusion/veterans-and-military-families.html
789 https://www.xenon.com.au/deep-learning-workshop 1682 https://www.xenon.com.au/careers Not Dl content https://www.xenon.com.au/careers
1417 https://docs.nvidia.com/deeplearning/dgx/mxnet-release-notes/index.html 294 https://www.nvidia.com/en-us/deep-learning-ai/industries/

421 http://www.nvidia.com/object/contact-information.html Not Dl content http://www.nvidia.com/object/contact-information.html
250 https://data.quora.com/First-Quora-Dataset-Release-Question-Pairs Not Dl content https://data.quora.com/First-Quora-Dataset-Release-Question-Pairs
1364 https://hollygrimm.com/bio Not Dl content https://hollygrimm.com/bio
315 https://www.nvidia.com/en-us/data-center/hpc 863 https://www.boozallen.com/markets/commercial-solutions/cyber-fusion-center.html Not Dl content https://www.boozallen.com/markets/commercial-solutions/cyber-fusion-center.html
616 https://developer.nvidia.com/cuda-education-training Not Dl content https://developer.nvidia.com/cuda-education-training
1288 https://dota2.gamepedia.com/Divine_Rapier Not Dl content https://dota2.gamepedia.com/Divine_Rapier
1107 https://www.youtube.com/user/nvidia Not Dl content https://www.youtube.com/user/nvidia
974 https://blog.openai.com/tag/greg-brockman Not Dl content https://blog.openai.com/tag/greg-brock

2489 https://developer.nvidia.com/taxonomy/term/1129 Not Dl content https://developer.nvidia.com/taxonomy/term/1129
755 https://courses.nvidia.com/courses/course-v1:DLI+L-AC-18+V1/about Not Dl content https://courses.nvidia.com/courses/course-v1:DLI+L-AC-18+V1/about
1686 https://www.xenon.com.au/sponsorship Not Dl content https://www.xenon.com.au/sponsorship
1934 http://www.nvidia.com/object/berkeley-request-a-call-for-dgx1.html Not Dl content http://www.nvidia.com/object/berkeley-request-a-call-for-dgx1.html
1798 https://www.nvidia.com/en-us/data-center/gpu-accelerated-applications/specfem3d-globe Not Dl content https://www.nvidia.com/en-us/data-center/gpu-accelerated-applications/specfem3d-globe
2330 https://www.pyimagesearch.com/category/dlib 1585 https://diuf.unifr.ch/main/hisdoc/divaservices Not Dl content https://diuf.unifr.ch/main/hisdoc/divaservices
1836 http://releases.ubuntu.com/16.04 Not Dl content http://releases.ubuntu.com/16.04
2183 https://www.dropbox.com/developers/refe

...

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


Not Dl content https://www.affirm.com
12737 http://www.parallelr.com/r-with-parallel-computing Unable to retrieve or parse page
HTTP Error 406: Not Acceptable
1919 https://www.xenon.com.au/product/nitro-t8 Not Dl content https://www.xenon.com.au/product/nitro-t8
5423 https://www.nvidia.com/en-au/deep-learning-ai/industries/ai-innovators 11626 https://southcentralus.dev.cognitive.microsoft.com/docs/services/5adf991815e1060e6355ad44/operations/587f2c6a154055056008f200/console Not Dl content https://southcentralus.dev.cognitive.microsoft.com/docs/services/5adf991815e1060e6355ad44/operations/587f2c6a154055056008f200/console
11210 https://www.reddit.com/r/MachineLearning/comments/46b8dz/what_does_debugging_a_deep_net_look_like/d04a9l6 Not Dl content https://www.reddit.com/r/MachineLearning/comments/46b8dz/what_does_debugging_a_deep_net_look_like/d04a9l6
10461 https://ngc.nvidia.com/catalog/landing Not Dl content https://ngc.nvidia.com/catalog/landing
9934 https://www.ibm.com/blogs/systems/t

12342 http://www.economist.com Not Dl content http://www.economist.com
9454 https://www.atlasvanlines.com Not Dl content https://www.atlasvanlines.com
7389 http://www.cisco.com Not Dl content http://www.cisco.com
10937 https://news.developer.nvidia.com/nvidia-rapids-accelerates-kubeflow-pipeline-with-gpus-on-kubernetes Not Dl content https://news.developer.nvidia.com/nvidia-rapids-accelerates-kubeflow-pipeline-with-gpus-on-kubernetes
12853 https://www.reddit.com/r/Python/comments/55nsck/automatically_grading_multiple_choice_testsexams/d8cu6kl Not Dl content https://www.reddit.com/r/Python/comments/55nsck/automatically_grading_multiple_choice_testsexams/d8cu6kl
11803 https://energydata.info/dataset/nigeria-nmis-education-facility-data-2014 Not Dl content https://energydata.info/dataset/nigeria-nmis-education-facility-data-2014
16795 https://blogs.skype.com/tag/translator Not Dl content https://blogs.skype.com/tag/translator
15977 http://ruder.io/lisbon-machine-learning-summer-school-hig

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


16553 https://cloud.withgoogle.com/partners Not Dl content https://cloud.withgoogle.com/partners
16302 https://raypack.ai/2018/12/merry-christmas-and-happy-new-year Not Dl content https://raypack.ai/2018/12/merry-christmas-and-happy-new-year
15776 https://www.ibm.com/blogs/systems/ibm-and-nvidia-further-collaboration-to-advance-open-source-gpu-acceleration Not Dl content https://www.ibm.com/blogs/systems/ibm-and-nvidia-further-collaboration-to-advance-open-source-gpu-acceleration
17149 https://money.cnn.com/data/world_markets/americas 12919 https://www.microsoft.com/en-us/research/people/srinath Not Dl content https://www.microsoft.com/en-us/research/people/srinath
11736 https://blogs.technet.microsoft.com/machinelearning/tag/azure-stream-analytics Not Dl content https://blogs.technet.microsoft.com/machinelearning/tag/azure-stream-analytics
8522 https://www.pyimageconf.com/code-of-conduct Not Dl content https://www.pyimageconf.com/code-of-conduct
16357 https://cognitive.uservoice.com/a

In [10]:
cur1 = conn.cursor()
cur1.execute('SELECT count(*) FROM Pages')

In [11]:
for row in cur1 : print(row)

(17280,)


In [17]:
#Cloud Object Storage setup    
import ibm_boto3
from ibm_botocore.client import Config
import os
import json
import warnings
import time

In [18]:
cos_credentials = {
  "apikey": "XXXXXXXXXX",
  "cos_hmac_keys": {
    "access_key_id": "XXXXXXXXXXXXXXXXXXXXXXXXXXX",
    "secret_access_key": "XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX"
  },
  "endpoints": "https://control.cloud-object-storage.cloud.ibm.com/v2/endpoints",
  "iam_apikey_description": "",
  "iam_apikey_name": "",
  "iam_role_crn": "",
  "iam_serviceid_crn": "",
  "resource_instance_id": ""
}

# You need to save the apikey and resource_instance_id to create the COS resource object later.
api_key = cos_credentials['apikey']
service_instance_id = cos_credentials['resource_instance_id']

# Define endpoint information.
service_endpoint = 'https://s3-api.us-geo.objectstorage.softlayer.net'
# Define the authorization endpoint.
auth_endpoint = 'https://iam.bluemix.net/oidc/token'
# Create a COS resource.
cos = ibm_boto3.resource ('s3',
                         ibm_api_key_id=api_key,
                         ibm_service_instance_id=service_instance_id,
                         ibm_auth_endpoint=auth_endpoint,
                         config=Config(signature_version='oauth'),
                         endpoint_url=service_endpoint)

In [19]:
def upload_file_cos(data_dir, local_file_name, key): 
    try: 
        bucket_obj = cos.Bucket('pagerank-donotdelete-pr-fghghg')
        with open(os.path.join(data_dir, local_file_name), 'rb') as data: 
            bucket_obj.upload_file(os.path.join(data_dir, local_file_name), key)
        print('{} is uploaded.'.format(local_file_name)) 
        for obj in bucket_obj.objects.all():
            print('Object key: {}'.format(obj.key))
            print('Object size (kb): {}'.format(obj.size/1024))
    except Exception as e:
        print(Exception, e)
    else:
        print('File Uploaded')

In [22]:
upload_file_cos('', 'spider.sqlite', 'spider1.sqlite')

spider.sqlite is uploaded.
Object key: notebook/spider_SkeH4phfE.ipynb
Object size (kb): 744.4072265625
Object key: spider.sqlite
Object size (kb): 6080.0
Object key: spider1.sqlite
Object size (kb): 366780.0
File Uploaded


In [23]:
upload_file_cos('', 'spider.sqlite-journal', 'spider1.sqlite-journal')

<class 'Exception'> [Errno 2] No such file or directory: 'spider.sqlite-journal'


# Run Page rank algorithm

In [13]:
my_bucket = cos.Bucket('pagerank-donotdelete-pr-dfgfgf')

In [14]:
my_bucket.download_file('spider.sqlite', 'spider.sqlite')

In [37]:
!ls

spider.js  spider.sqlite  sprank.py


In [13]:
#conn = sqlite3.connect('spider.sqlite')
cur = conn.cursor()

# Find the ids that send out page rank - we only are interested
# in pages in the SCC that have in and out links
cur.execute('''SELECT DISTINCT from_id FROM Links''')
from_ids = list()
for row in cur: 
    from_ids.append(row[0])

# Find the ids that receive page rank 
to_ids = list()
links = list()
cur.execute('''SELECT DISTINCT from_id, to_id FROM Links''')
for row in cur:
    from_id = row[0]
    to_id = row[1]
    if from_id == to_id : continue
    if from_id not in from_ids : continue
    if to_id not in from_ids : continue
    links.append(row)
    if to_id not in to_ids : to_ids.append(to_id)

# Get latest page ranks for strongly connected component
prev_ranks = dict()
for node in from_ids:
    cur.execute('''SELECT new_rank FROM Pages WHERE id = ?''', (node, ))
    row = cur.fetchone()
    prev_ranks[node] = row[0]

many = 50 #no.of iterations

# Sanity check
if len(prev_ranks) < 1 : 
    print("Nothing to page rank.  Check data.")
    quit()

# Lets do Page Rank in memory so it is really fast
for i in range(many):
    # print prev_ranks.items()[:5]
    next_ranks = dict();
    total = 0.0
    for (node, old_rank) in list(prev_ranks.items()):
        total = total + old_rank
        next_ranks[node] = 0.0
    # print total

    # Find the number of outbound links and sent the page rank down each
    for (node, old_rank) in list(prev_ranks.items()):
        # print node, old_rank
        give_ids = list()
        for (from_id, to_id) in links:
            if from_id != node : continue
           #  print '   ',from_id,to_id

            if to_id not in to_ids: continue
            give_ids.append(to_id)
        if ( len(give_ids) < 1 ) : continue
        amount = old_rank / len(give_ids)
        # print node, old_rank,amount, give_ids
    
        for id in give_ids:
            next_ranks[id] = next_ranks[id] + amount
    
    newtot = 0
    for (node, next_rank) in list(next_ranks.items()):
        newtot = newtot + next_rank
    evap = (total - newtot) / len(next_ranks)

    # print newtot, evap
    for node in next_ranks:
        next_ranks[node] = next_ranks[node] + evap

    newtot = 0
    for (node, next_rank) in list(next_ranks.items()):
        newtot = newtot + next_rank

    # Compute the per-page average change from old rank to new rank
    # As indication of convergence of the algorithm
    totdiff = 0
    for (node, old_rank) in list(prev_ranks.items()):
        new_rank = next_ranks[node]
        diff = abs(old_rank-new_rank)
        totdiff = totdiff + diff

    avediff = totdiff / len(prev_ranks)
    print(i+1, avediff)

    # rotate
    prev_ranks = next_ranks

# Put the final ranks back into the database
print(list(next_ranks.items())[:5])
cur.execute('''UPDATE Pages SET old_rank=new_rank''')
for (id, new_rank) in list(next_ranks.items()) :
    cur.execute('''UPDATE Pages SET new_rank=? WHERE id=?''', (new_rank, id))
conn.commit()
cur.close()

1 0.8687244001781042
2 0.37793396321883693
3 0.17247906492399437
4 0.09707382673793248
5 0.06438603983724693
6 0.047190598963056364
7 0.0366537206011413
8 0.02948279070345955
9 0.024437400993381
10 0.02092691787825796
11 0.0183122713032182
12 0.01630226672911465
13 0.014773666681123713
14 0.013554105796335411
15 0.012602959641957404
16 0.011974489135021752
17 0.01169195747926793
18 0.0115766392376785
19 0.011487445450387506
20 0.011415358941963225
21 0.011344050307392524
22 0.011282870764362731
23 0.011220792500327308
24 0.011166908655641287
25 0.011111083044592416
26 0.011062319792726392
27 0.011011000770453005
28 0.010965943441283666
29 0.010918343184875834
30 0.010876621021949737
31 0.010832783039223512
32 0.010794579357856878
33 0.010753211111654296
34 0.010716576005831804
35 0.010676767728213158
36 0.01064140969426713
37 0.010602901582313421
38 0.010568607969244042
39 0.010531205572251458
40 0.010497817892540506
41 0.010461373103436271
42 0.010428770442048162
43 0.0103931686714359

# Visualize current top pages 

In [14]:
#conn = sqlite3.connect('spider.sqlite')
cur = conn.cursor()

print("Creating JSON output on spider.js...")
howmany = 50

cur.execute('''SELECT COUNT(from_id) AS inbound, old_rank, new_rank, id, url 
    FROM Pages JOIN Links ON Pages.id = Links.to_id
    WHERE html IS NOT NULL AND ERROR IS NULL
    GROUP BY id ORDER BY id,inbound''')

fhand = open('spider.js','w')
nodes = list()
maxrank = None
minrank = None
for row in cur :
    nodes.append(row)
    rank = row[2]
    if maxrank is None or maxrank < rank: maxrank = rank
    if minrank is None or minrank > rank : minrank = rank
    if len(nodes) > howmany : break

if maxrank == minrank or maxrank is None or minrank is None:
    print("Error - please run sprank.py to compute page rank")
    quit()

fhand.write('spiderJson = {"nodes":[\n')
count = 0
map = dict()
ranks = dict()
for row in nodes :
    if count > 0 : fhand.write(',\n')
    # print row
    rank = row[2]
    rank = 19 * ( (rank - minrank) / (maxrank - minrank) ) 
    fhand.write('{'+'"weight":'+str(row[0])+',"rank":'+str(rank)+',')
    fhand.write(' "id":'+str(row[3])+', "url":"'+row[4]+'"}')
    map[row[3]] = count
    ranks[row[3]] = rank
    count = count + 1
fhand.write('],\n')

cur.execute('''SELECT DISTINCT from_id, to_id FROM Links''')
fhand.write('"links":[\n')

count = 0
for row in cur :
    # print row
    if row[0] not in map or row[1] not in map : continue
    if count > 0 : fhand.write(',\n')
    rank = ranks[row[0]]
    srank = 19 * ( (rank - minrank) / (maxrank - minrank) ) 
    fhand.write('{"source":'+str(map[row[0]])+',"target":'+str(map[row[1]])+',"value":3}')
    count = count + 1
fhand.write(']};')
fhand.close()
cur.close()

Creating JSON output on spider.js...


In [15]:
!ls

spider.js  spider.sqlite


In [24]:
upload_file_cos('', 'spider.js', 'spider.js')

spider.js is uploaded.
Object key: data_flow_run/data_flow_run_log__vZa3ARDRMqMvZVjQEm47w
Object size (kb): 0.421875
Object key: data_flow_run/data_flow_run_origin_sR5HiVf4SfS-9ZRL_Nfhfg
Object size (kb): 2.19921875
Object key: notebook/spider_SkeH4phfE.ipynb
Object size (kb): 544.1611328125
Object key: notebook/spider_copy_1_Nkc8R9cMI.ipynb
Object size (kb): 743.755859375
Object key: spider.js
Object size (kb): 15.1728515625
Object key: spider.sqlite
Object size (kb): 6080.0
Object key: spider1.sqlite
Object size (kb): 366780.0
Object key: sprank.py
Object size (kb): 2.9853515625
File Uploaded


In [22]:
!cat spider.js

spiderJson = {"nodes":[
{"weight":8,"rank":1.3452420123866975, "id":1, "url":"https://blog.floydhub.com/spinning-up-with-deep-reinforcement-learning"},
{"weight":101,"rank":14.46212163069049, "id":2, "url":"https://blog.floydhub.com"},
{"weight":39,"rank":9.953418183173474, "id":4, "url":"https://blog.floydhub.com/write-for-floydhub"},
{"weight":3,"rank":2.125268913190739, "id":7, "url":"https://blog.openai.com/spinning-up-in-deep-rl"},
{"weight":46,"rank":6.4822798537800885, "id":8, "url":"https://spinningup.openai.com/en/latest/spinningup/spinningup.html"},
{"weight":7,"rank":0.39316730020759344, "id":9, "url":"http://karpathy.github.io/2016/05/31/rl"},
{"weight":43,"rank":1.1561835971037082, "id":11, "url":"https://lilianweng.github.io/lil-log/2018/04/08/policy-gradient-algorithms.html"},
{"weight":10,"rank":0.49602731169844916, "id":12, "url":"http://amid.fish/reproducing-deep-rl"},
{"weight":1,"rank":0.0, "id":17, "url":"https://medium.com/mlreview/speeding-up-dqn-on-pytorch-solvi